<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/RecercaExtra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import torch
import cv2
import csv
import random
import numpy as np
import ast
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl
import json
import sys
import ast
import shutil

from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io
from google.colab import drive

# ORIGEN = '/content/drive/MyDrive/HAM10000/skin-cancer-mnist-ham10000/'
DOWNLOAD = False
PUT = False
RUN = 22

TESTING_FEATURES_FILE = '/content/drive/MyDrive/Runs/Ham10000Resnet152Multiclass/'+str(RUN)+'/Features/testing_features.txt'
TRAINING_FEATURES_FILE = '/content/drive/MyDrive/Runs/Ham10000Resnet152Multiclass/'+str(RUN)+'/Features/training_features.txt'
HAIRIMG = '/content/drive/MyDrive/Runs/Ham10000Resnet152Multiclass/'+str(RUN)+'/HairFeature/'


if PUT:
  if os.path.exists(HAIRIMG):
      shutil.rmtree(HAIRIMG)

  os.makedirs(HAIRIMG,exist_ok=True)

drive.mount('/content/drive')

if DOWNLOAD:
  !rm -rf /content/sample_data/*

  !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

  !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/


# IMAGE_PATHS1 = glob('/content/sample_data/HAM10000_images_part_1/*.jpg')
# IMAGE_PATHS2 = glob('/content/sample_data/HAM10000_images_part_2/*.jpg')
# IMAGE_PATHS3 = glob('/content/sample_data/ham10000_images_part_1/*.jpg')
# IMAGE_PATHS4 = glob('/content/sample_data/ham10000_images_part_2/*.jpg')
# IMAGE_PATHS = IMAGE_PATHS1 + IMAGE_PATHS2 + IMAGE_PATHS3 + IMAGE_PATHS4
# PATH_TRAINING_DISTANCES = "/content/drive/MyDrive/Features/Embeddings/embeddings_pesos_resnet152multiclass_3.pt.txt" #embeddings training
# DISTANCES_IMAGES = "/content/drive/MyDrive/Features/Dist/1/distances.txt" #distàncies de totes les imatges de test

IMG_PATH = '/content/sample_data/HAM10000_images_part_1/'

IMATGES_INTERES = ["ISIC_0027790.jpg","ISIC_0027010.jpg","ISIC_0025415.jpg","ISIC_0026286.jpg","ISIC_0025606.jpg","ISIC_0028732.jpg"
,"ISIC_0026879.jpg","ISIC_0025466.jpg","ISIC_0025746.jpg","ISIC_0026193.jpg","ISIC_0026947.jpg","ISIC_0028777.jpg","ISIC_0024803.jpg","ISIC_0026292.jpg"
,"ISIC_0027970.jpg","ISIC_0028921.jpg","ISIC_0024891.jpg","ISIC_0026315.jpg","ISIC_0026392.jpg","ISIC_0027199.jpg","ISIC_0026127.jpg"
,"ISIC_0026303.jpg","ISIC_0029097.jpg","ISIC_0025282.jpg","ISIC_0028642.jpg","ISIC_0027295.jpg","ISIC_0025216.jpg","ISIC_0028424.jpg","ISIC_0025526.jpg"
,"ISIC_0026184.jpg","ISIC_0024779.jpg","ISIC_0026998.jpg","ISIC_0024342.jpg","ISIC_0024411.jpg","ISIC_0026615.jpg","ISIC_0026651.jpg"] #imatges amb els patrons que cercam

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if PUT:
  for i in IMATGES_INTERES:
    shutil.copy(IMG_PATH+i,HAIRIMG)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
features = []
def guarda_features_hairy():
  for j in IMATGES_INTERES:
    with open(TESTING_FEATURES_FILE ,newline = '') as ftst:
      test_features = csv.DictReader(ftst,delimiter=',')

      for row in test_features:
        if row['Image Name'] == j:
          features.append(ast.literal_eval(row['FC Input']))

    with open(TRAINING_FEATURES_FILE ,newline = '') as ftr:
      tr_features = csv.DictReader(ftr,delimiter = ',')

      for row in tr_features:
        if row['Image Name'] == j:
          features.append(ast.literal_eval(row['FC Input']))



In [59]:
# Supón que IMAGE_PATHS ya contiene todas tus rutas
# for path in IMAGE_PATHS:  # Puedes cambiar el número de imágenes que deseas ver
#     img = Image.open(path)
#     clear_output(wait=True)  # Limpia la celda para mostrar la siguiente imagen
#     display(img)
#     print(path)
#     time.sleep(5)  # Espera 2 segundos


In [60]:
# !mkdir -p ~/.kaggle
# !mv /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [61]:
# print(vector_features)